In [ ]:
import uwacan
import uwacan.sensors
from pathlib import Path

import pendulum
import numpy as np
import xarray as xr

from importlib import reload
reload(uwacan.sensors)
reload(uwacan.recordings)


In [ ]:
base_path = Path('F:\Tern Island Vinga')
sensors = uwacan.sensors.sensor_array(
    uwacan.sensors.sensor(label='SoundTrap 1', depth=35, sensitivity=-188.2),
    uwacan.sensors.sensor(label='SoundTrap 2', depth=38, sensitivity=-188.9),
    uwacan.sensors.sensor(label='SoundTrap 3', depth=36, sensitivity=-185.4),
    uwacan.sensors.sensor(label='Sylence 1', depth=40, sensitivity=-188.2),
    uwacan.sensors.sensor(label='Sylence 2', depth=25, sensitivity=-188.2),
)
sensors

In [ ]:
soundtrap_1 = uwacan.recordings.SoundTrap.read_folder(
    base_path / 'Autonomous Recorders' / 'IVL SoundTrap 1',
    sensor=sensors.sel(sensor='SoundTrap 1'),
)
soundtrap_2 = uwacan.recordings.SoundTrap.read_folder(
    base_path / 'Autonomous Recorders' / 'IVL SoundTrap 2',
    sensor=sensors.sel(sensor='SoundTrap 2'),
)
soundtrap_3 = uwacan.recordings.SoundTrap.read_folder(
    base_path / 'Autonomous Recorders' / 'IVL SoundTrap 3',
    sensor=sensors.sel(sensor='SoundTrap 3'),
)

sylence_1 = uwacan.recordings.SylenceLP.read_folder(
    base_path / 'Autonomous Recorders' / 'IVL Sylence 1',
    sensor=sensors.sel(sensor='Sylence 1'),
)


In [ ]:
test = uwacan.recordings.AudioFileRecording.read_folder(
    folder=base_path / 'Manual Recordings' / 'Fireface',
    start_time_parser=lambda filename: pendulum.from_format(filename.stem, 'YYYY-MM-DD_HH-mm-ss'),
    sensor=sensors.sel(sensor=['SoundTrap 1', 'SoundTrap 2', 'SoundTrap 3'])
)
test.sampling.subwindow(start="2023-08-30 11:39:08", duration=10).time_data

In [ ]:
self = test.sampling.subwindow(start="2023-08-30 11:39:08", duration=10)
data = self.read_data()


In [ ]:
cabled_sensors = uwacan.sensors.sensor_array(
    uwacan.sensors.sensor('Colmar 155', sensitivity=-166),
    uwacan.sensors.sensor('Colmar 156', sensitivity=-166),
    uwacan.sensors.sensor('Colmar 157', sensitivity=-166),
)

In [ ]:
recordings = uwacan.recordings.MultichannelAudioFileRecording.read_folder(
    base_path / 'Manual Recordings' / 'Fireface',
    sensor=cabled_sensors,
    start_time_parser=lambda file: pendulum.from_format(file.stem, 'YYYY-MM-DD_HH-mm-ss'),
)

In [ ]:
recordings = uwacan.recordings.MultichannelAudioFileRecording.read_folder(
    base_path / 'Manual Recordings' / 'Fireface',
    sensor=cabled_sensors.isel(sensor=slice(1)),
    start_time_parser=lambda file: pendulum.from_format(file.stem, 'YYYY-MM-DD_HH-mm-ss'),
    channel=[1],
    one_recorder_per_file=False,
)

In [ ]:
recordings.sampling.subwindow(start="2023-08-30 11:39:08", duration=10).time_data

In [ ]:
latitude = xr.DataArray([[58, 58.5, 59]], dims=('other', 'time'), coords={'time': [1,2,3]})
longitude = xr.DataArray([[11, 11.5, 11.5]], dims=('other', 'time'), coords={'other': [15]})
track = xr.Dataset(dict(
    latitude=latitude,
    longitude=longitude,
))

ref = xr.Dataset(dict(latitude=58.5, longitude=12))


distances = uwacan.positional.distance_between(ref, track)
track.assign(distance=distances).isel(distances.argmin(...))


In [ ]:
track.isel(distances.argmin(...))

In [ ]:
def format_coordinates(*args, format='minutes', precision=None):
    position = uwacan.positional.position(*args)
    latitude = np.atleast_1d(position.latitude.values)
    longitude = np.atleast_1d(position.longitude.values)
    def ns(lat):
        return 'N' if lat > 0 else 'S'
    def ew(lon):
        return 'E' if lon > 0 else 'W'

    if format.lower()[:3] == 'deg':
        if precision is None:
            precision = 6

        def format(lat, lon):
            lat = f"{abs(lat):.{precision}f}°{ns(lat)}"
            lon = f"{abs(lon):.{precision}f}°{ew(lon)}"
            return lat + " " + lon

    elif format.lower()[:3] == 'min':
        if precision is None:
            precision = 4

        def format(lat, lon):
            latdeg, latmin = np.divmod(abs(lat) * 60, 60)
            londeg, lonmin = np.divmod(abs(lon) * 60, 60)
            lat = f"{abs(latdeg):.0f}°{latmin:.{precision}f}'{ns(lat)}"
            lon = f"{abs(londeg):.0f}°{lonmin:.{precision}f}'{ew(lon)}"
            return lat + " " + lon
    elif format.lower()[:3] == 'sec':
        format = 'sec'
        if precision is None:
            precision = 2

        def format(lat, lon):
            latdeg, latmin = np.divmod(abs(lat) * 60, 60)
            londeg, lonmin = np.divmod(abs(lon) * 60, 60)
            latmin, latsec = np.divmod(latmin * 60, 60)
            lonmin, lonsec = np.divmod(lonmin * 60, 60)
            lat = f"""{abs(latdeg):.0f}°{latmin:.0f}'{latsec:.{precision}f}"{ns(lat)}"""
            lon = f"""{abs(londeg):.0f}°{latmin:.0f}'{lonsec:.{precision}f}"{ew(lon)}"""
            return lat + " " + lon

    formatted = [
        format(lat, lon)
        for lat, lon in zip(latitude, longitude)
    ]
    if len(formatted) == 1:
        return formatted[0]
    return formatted


In [ ]:
print(format_coordinates([58.00001, 59], [11.00094, 23.45], format='min'))

In [ ]:
uwacan.positional.distance_between(
    uwacan.positional.Position(58, 0),
    # uwacan.positional.Position(58, 0.0001, 0, 0),
    # uwacan.positional.Position(58.000001, 0, 0),
    uwacan.positional.Position(58, 0, 0.01, 0, 0, 0),
)

In [ ]:
reload(uwacan.positional)

In [ ]:
p = uwacan.positional.Position(58, 0)
uwacan.positional.shift_position(p, 1, 0).minutes

In [ ]:
def coordinate_minutes(position):
    latdeg, latmin = np.divmod(position.latitude * 60, 60)
    londeg, lonmin = np.divmod(position.longitude * 60, 60)
    return xr.Dataset(dict(
        latitude_degrees=latdeg,
        latitude_minutes=latmin,
        longitude_degrees=londeg,
        longitude_minutes=lonmin,
    ))

In [ ]:
np.div

In [ ]:
lat = 57.6931022
lon = 11.974318

coordinate_minutes(xr.Dataset({'latitude': lat, 'longitude': lon}))

In [ ]:
p = uwacan.positional.Position(57.6931022, 11.974318)
display(p.minutes)
display(p.seconds)

display(uwacan.positional.Position(57, 41.586132, 11, 58.45908) == p)
display(uwacan.positional.Position(57, 41, 35.17, 11, 58, 27.54).distance_to(p))


In [ ]:
recordings[3].num_channels

In [ ]:
uwacan.dB(sylence_1.sampling.subwindow(start=True, duration=10).time_data, power=False)


In [ ]:
soundtraps = uwacan.recordings.RecordingArray(soundtrap_1, soundtrap_2, soundtrap_3)


In [ ]:
soundtraps.sampling.subwindow(start=True, duration=100).time_data


In [ ]:
soundtraps.num_channels


In [ ]:
soundtrap_1.files[0].__class__.mro()


In [ ]:
array = uwacan.recordings.RecordingArray(
    soundtrap_1,
    soundtrap_2,
    soundtrap_3,
    sylence_1,
)


In [ ]:
uwacan.recordings.SequentialAudioFileRecorder.RecordedFile


In [ ]:
time_data = array.sampling.subwindow(start=True, duration=1).time_data


In [ ]:
import numpy as np


In [ ]:
np.ndim(array.sampling.rate)


In [ ]:
soundtrap_1 = uwacan.recordings.SoundTrap(
    folder=base_path / 'Autonomous Recorders' / 'IVL SoundTrap 1',
    serial_number=5305,
    calibration=188.2,
)
soundtrap_2 = uwacan.recordings.SoundTrap(
    folder=base_path / 'Autonomous Recorders' / 'IVL SoundTrap 2',
    serial_number=5300,
    calibration=188.9,
)
soundtrap_3 = uwacan.recordings.SoundTrap(
    folder=base_path / 'Autonomous Recorders' / 'IVL SoundTrap 3',
    serial_number=5485,
    calibration=185.4,
)


In [ ]:
elvy.soundtraps = uwacan.recordings.ColocatedHydrophoneArray(
    uwacan.recordings.SoundTrap(
        os.path.join(elvy.path, 'Soundtraps'),
        serial_number=5305,
        depth=6,
        calibration=188.2,
        time_compensation=3.836,
        position=(57, 35.424, 11, 48.350),
        # position=uwacan.positional.Position(57, 35.424, 11, 48.350),
    ),
    uwacan.recordings.SoundTrap(
        os.path.join(elvy.path, 'Soundtraps'),
        serial_number=5300,
        depth=11,
        calibration=188.9,
        time_compensation=0.47,
        position=(57, 35.424, 11, 48.350),
        # position=uwacan.positional.Position(57, 35.424, 11, 48.350),
    ),
    uwacan.recordings.SoundTrap(
        os.path.join(elvy.path, 'Soundtraps'),
        serial_number=5485,
        depth=16,
        calibration=185.4,
        time_compensation=-0.196,
        position=(57, 35.424, 11, 48.350)
        # position=uwacan.positional.Position(57, 35.424, 11, 48.350),
    ),
)
